In [1]:
import cv2
import pytesseract
#import csv
import numpy as np
import pandas as pd
from ultralytics import YOLO
import textract
#from PIL import Image
#import matplotlib.pyplot as plt
import keras_ocr
import Levenshtein


In [2]:
#Tesseract configuration
custom_words = 'f_dictionnary_v1.txt'
config = f'--psm 6 --user-words {custom_words}'
pipeline = keras_ocr.pipeline.Pipeline()

Looking for C:\Users\Flam\.keras-ocr\craft_mlt_25k.h5
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Looking for C:\Users\Flam\.keras-ocr\crnn_kurapan.h5


In [171]:
#cleaning strings
def strip_string(my_string):
    replacements = [('!', ''), ('?', ''),('|',''),("[",""),("]",""),("/",""),('\\',""),('\r',''),('\'',''),('"',''),('',''),("\u200f",""),("\u200e",""),("{",""),("}",""),("%",""),("~",""),(",","."),(':','.'),(";","."),('°',''),('+',''),('(',''),(')','')]
    for char, replacement in replacements:
        if char in my_string:
            my_string = my_string.replace(char, replacement)
    return my_string

def strip_dates(my_string):
    replacements = [('!', ''), ('?', ''),('|',''),("[",""),("]",""),("%",""),("\u200f",""),("\u200e",""),("{",""),("}",""),("&","")]
    for char, replacement in replacements:
        if char in my_string:
            my_string = my_string.replace(char, replacement)
    return my_string

def prepare(my_string,flag):
    if (flag!='paragraph2'):
        tmp=strip_string(my_string)
    else:
        tmp=strip_dates(my_string)
    tmp=tmp.split("\n")
    while("" in tmp):
        tmp.remove("")
    tmp2=tmp
    tmp3=tmp
    if (flag=='tab'):
        tmp2 = [i.replace(' ','') for i in tmp]
        tmp3 = [i.replace('o','0') for i in tmp2]
        #print(tmp3)
        tmp4=' '.join(tmp3)
        return (tmp4)
    return(tmp3)

def remove_substring_except_last(string, substring):
    # Replace all occurrences of the substring except the last one with an empty string
    new_string = string.replace(substring, "", string.count(substring)-1)
    # Split the string at the last occurrence of the substring
    parts = new_string.rsplit(substring, 1)
    # Join the parts back together with the substring
    return substring.join(parts)

def reglage_montant(money):
    for i in range(len(money)):
        money[i]=remove_substring_except_last(money[i],'.')
    return (money)

def get_numbers(test_string):
    letter = [x for x in test_string]
    res = [i for i in letter if i.isdigit()]
    result=''.join(res)
    if (len(result)>0):
        return result
    else:
        return (test_string)
def containsNumber(value):
    for character in value:
        if character.isdigit():
            return True
    return False
def number_correction(string):
    corrected=[]
    replacements = [("a","0"),("p","0"),("q","0"),("d","0"),("o","0"),("O","0"),('e','6')]
    X=string.split(" ")
    for i in X:
        letters=i
        for char, replacement in replacements:
            if char in letters:
                letters = letters.replace(char, replacement)
        corrected.append(letters)
    ss=" ".join(corrected)
    return (ss)

In [4]:
#Correcting words with help of dictionnary
def find_closest_word(word, dictionary_file):
    smallest_distance = float('inf')
    closest_word = ""

    with open(dictionary_file,encoding='utf-8') as f:
        words = [line.strip() for line in f]

    for dict_word in words:
        distance = Levenshtein.distance(word, dict_word)
        if distance < smallest_distance:
            smallest_distance = distance
            closest_word = dict_word

    return closest_word

In [191]:
#function to enhance the contrast of image for better ocr result
def enhance(im):
    img = cv2.imread(im)
    # converting to LAB color space
    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    # Applying CLAHE to L-channel
    # feel free to try different values for the limit and grid size:
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((cl,a,b))

    # Converting image from LAB Color model to BGR color spcae
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    # Stacking the original image with the enhanced image
    #result = np.hstack((img, enhanced_img))
    #cv2.imshow('Result', result)
    #cv2.waitKey(0)
    return enhanced_img

## Treating type 1 invoices

In [107]:
#cleaning and extracting data from para 1
def extract_para1(para1):
    parag=prepare(para1[0],'paragraph1')
    LOIs=[parag[0],parag[1]]
    #print(LOIs)
    for LOI in LOIs:
        loi=LOI.split()
        for item in loi:
            try:
                n_facture=int(item)
            except:
                n_facture=0
            if(n_facture > 0 ):
                return (n_facture)
    return(0)
    #print(LOI)

In [3]:
#cleaning and extracting data from para 2
def fix_date(date):
    return find_closest_word(date,'dates.txt')
def extract_para2(para2):
    parag=prepare(para2[0],'paragraph2')
    clean=[]
    for i in parag:
        if len(i)>3:
            clean.append(i)
    #print(clean)
    mois=clean[0].split()
    ref=clean[2].split()
    #print(ref)
    for item in ref:
        try:
            ref_=int(item)
        except:
            ref_=0
        if(ref_ != 0 ):
            break
    for item in mois:
        res = any(chr.isdigit() for chr in item)
        if (res):
            mois_=item
            break
    mois_=fix(mois_)
    #print(ref_)
    #print(mois_)
    return([mois_,ref_])

In [7]:
# extracting bounding boxes cordinates and their labels
def boxes_labels(results,x,y,h,w,c):
    bbox=results[0].boxes.boxes
    bbox=bbox.cpu().numpy()
    boxes=results[0].boxes.xyxy
    for i,box in enumerate(boxes):
        box=box.cpu().numpy()
        print("fragment ",i," :",box)
        x.append(int(box[0]))
        y.append(int(box[1]))
        h.append(int(box[3]-y[i]))
        w.append(int(box[2]-x[i]))
        c.append(bbox[i][5])

In [8]:
#storing ROIs in a liste
def crop_image(x,y,h,w):
    cropped=[]
    for (X,Y,H,W) in zip(x,y,h,w):
        cropped.append(img[Y:Y+H, X:X+W])
    print("cropped images: ",len(cropped))
    return (cropped)

In [9]:
#to avoid the presence of multi instance from classes
def singilarity(cropped,c,classes,images):
   # images=[]
    for (img,C) in zip(cropped,c):
        try:
            i=classes.index(C)
        except:
            i=-1
        if i<0 :
            classes.append(C)
            images.append(img)
        else :
            if len(img)>len(images[i]) :
                images[i]=img
    print('number of images:',len(images),'number of classes',len(classes))
            

In [10]:
#Return the best result from three ocr results
def best_res(string1,string2,string3):
    # Define the three OCR strings
    flag=False
    if (string1==''):
        flag=True
        string1='0123456789.'
    if(string2==''):
        if (flag==False):
            flag=True
            string2='0123456789.'
    if(string3==''):
        if (flag==False):
            flag=True
            string3='0123456789.'
    ocr_strings = [string1,string2,string3]
    #print('st1:',string1,'st2:',string2,'st3:',string3)
    # Calculate the average Levenshtein distance between each OCR string and the other two strings
    avg_distances = []
    for i in range(len(ocr_strings)):
        total_distance = 0
        for j in range(len(ocr_strings)):
            if i != j:
                total_distance += Levenshtein.distance(ocr_strings[i], ocr_strings[j])
        avg_distance = total_distance / 2
        avg_distances.append(avg_distance)

    # Determine the most accurate result based on the average distances
    #if min(avg_distances) < 4:
    index = avg_distances.index(min(avg_distances))
    if len(ocr_strings[index])>0:
        return (ocr_strings[index])
    else:
        return('error')
    #else:
        #return ('not reliable')


In [155]:
def perform_ocr(ocr_im,f,check=False):
    res_keras=[]
    corrected=[]
    res_tes=[]
    tt=cv2.imread(ocr_im)
    if (f == 'ar'):
        res_tess=pytesseract.image_to_string(tt,config=config,lang='ara')
        #res_tes=prepare(res_tes,'tab')
        print(res_tess)
        res_tess=res_tess.split('\n')
        for x in res_tess:
            if len(x)>1:
                res_tes.append(x)
        print(res_tes)
        corrected=[]
        for t in res_tes:
            line=''
            tmp=t.split(" ")
            for i in tmp :
                if(containsNumber(i)):
                    line+=i+' '
                else :
                    line+=find_closest_word(i, "arabic.txt")+' '
            corrected.append(line)
        #print(corrected)
        return (corrected)
    elif (f == '*'):
        res_tes=pytesseract.image_to_string(tt,config=config,lang='ara')
        res_tes=prepare(res_tes,'tab')
        if ("تقدير" not in res_tes):
            res_opus=textract.process(ocr_im,method='ocropus')
            res_opus=res_opus.decode("utf-8")
            res_opus=prepare(res_opus,'tab')
            predictions = pipeline.recognize([tt])
            for prediction in predictions[0]:
                res_keras=' '.join(prediction[0])
                res_keras=number_correction(res_keras)
            best=best_res(res_keras,res_tes,res_opus)
            best_s=best.split()
            return (best_s)
        else :
            return (res_tes.split())
    else:
        res_tes=pytesseract.image_to_string(tt,config=config)
        res_tes=prepare(res_tes,'tab')
        res_tes=number_correction(res_tes)
        print(res_tes)
        res_opus=textract.process(ocr_im,method='ocropus')
        res_opus=res_opus.decode("utf-8")
        res_opus=prepare(res_opus,'tab')
        res_opus=number_correction(res_opus)
        print(res_opus)
        predictions = pipeline.recognize([tt])
        for prediction in predictions[0]:
            res_keras.append(prepare(prediction[0],'tab'))
        res_keras=' '.join(res_keras)
        if (check==True):
            if("." not in res_keras):
                res_keras=''
        print(res_keras)
        best=best_res(res_tes,res_opus,res_keras)
        best_s=best.split()
        return (best_s)

In [1]:
#Loading the image on which we are going to perfom ocr
image='D1/18.jpg'
img=cv2.imread(image,0)
img.shape

NameError: name 'cv2' is not defined

In [99]:
#results2_tab = cellss(image)

In [160]:
# Load a model
cells = YOLO("weights/cells_v5.pt")# load the table cells detection model
global_model= YOLO("weights/global_v2.pt")    #load the global detection model

# detecting the 3 main region of interests within the invoice
results_global = global_model(image)  # predict on an image

Ultralytics YOLOv8.0.22  Python-3.9.12 torch-1.13.1 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs
image 1/1 C:\Users\Flam\Desktop\Stage WCT\D1\18.jpg: 640x480 1 paragraph1, 1 paragraph2, 1 table, 21.0ms
Speed: 3.0ms pre-process, 21.0ms inference, 42.0ms postprocess per image at shape (1, 3, 640, 640)


In [161]:
#Extracting bounding boxes and ROI from the global model
x=[]
y=[]
h=[]
w=[]
c=[]
cropped=[]
boxes_labels(results_global,x,y,h,w,c)
cropped=crop_image(x,y,h,w)
classes=[]
images=[]
singilarity(cropped,c,classes,images)


fragment  0  : [         20         173         652         418]
fragment  1  : [         43         484        1240         782]
fragment  2  : [        661         168        1233         414]
cropped images:  3
number of images: 3 number of classes 3


In [162]:
#performing OCR on the two paragraphs above the table
ocr_results=[]
for C,img in zip(classes,images):
    if (C != 2):
        ocr_results.append([pytesseract.image_to_string(img,config=config),int(C)])
    else :
        cv2.imwrite('sliced.jpg', img)

In [163]:
for i in ocr_results:
    if (i[1]==1):
        n_facture=extract_para1(i)
    else :
        mois_ref=extract_para2(i)

In [164]:
n_facture

834102022404

In [167]:
img=cv2.imread('sliced.jpg')
results_tab = cells(img)
#results_tab2 = cells(image)


0: 160x640 1 consommation, 1 libele, 1 montant, 1 month, 97.1ms
Speed: 363.4ms pre-process, 97.1ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
image 1/1 C:\Users\Flam\Desktop\Stage WCT\D1\18.jpg: 640x480 1 consommation, 1 libele, 1 montant, 1 month, 194.4ms
Speed: 1.9ms pre-process, 194.4ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)


In [166]:
#Extracting bounding boxes and ROI from the table model
x=[]
y=[]
h=[]
w=[]
c=[]
cropped=[]
boxes_labels(results_tab,x,y,h,w,c)
cropped=crop_image(x,y,h,w)
classes=[]
images=[]
singilarity(cropped,c,classes,images)


fragment  0  : [        273          59         426         203]
fragment  1  : [        472          59         602         209]
fragment  2  : [        878          56         945         198]
fragment  3  : [        954          54        1187         200]
cropped images:  4
number of images: 4 number of classes 4


In [176]:
#storing cropped images locally
exceptions=[0,0,0,0,0]
try:
    cv2.imwrite('mont.jpg', images[classes.index(2)])
except:
    exceptions[2]=1
try:
    cv2.imwrite('cons.jpg', images[classes.index(0)])
except:
    exceptions[0]=1
try:
    cv2.imwrite('indexe.jpg', images[classes.index(4)])
except:
    exceptions[4]=1
try:
    cv2.imwrite('month.jpg', images[classes.index(3)])
except:
    exceptions[3]=1
try:
    
    cv2.imwrite('title.jpg', images[classes.index(1)])
    cv2.imwrite('title2.jpg',enhance('title.jpg'))
except:
    exceptions[1]=1

In [169]:
exceptions

[0, 0, 0, 0, 1]

In [184]:
if(exceptions[4]==0):
    indexe=perform_ocr('indexe.jpg','*')
else:
    indexe=['error', 'error']
if(exceptions[0]==0):
    consommation=perform_ocr('cons.jpg','en')
else:
    consommation=['0' ,'0']
if(exceptions[1]==0):
    titles=perform_ocr('title2.jpg','ar')
else:
    titles=['error' ,'error']
if(exceptions[3]==0):
    month=perform_ocr('month.jpg','en')
else:
    month=['error','error'] 
if(exceptions[2]==0):
    montant=perform_ocr('mont.jpg','en',check=True)
else:
    montant=['0.000','0.000']

21557
21557
21557
*5 33 643155
ءة 33 53000

['*5 33 643155', 'ءة 33 53000']
006 006
00é 006
096 096
8387.664 46.200
8387.664 46.200



In [188]:
month

['006', '006']

In [189]:
s_title =[]
s_month=[]
s_indexe =[]
s_consommation =[]
s_montant =[]
i1,i2,i3,i4,i5 = len(titles)-1,len(month)-1,len(indexe)-1,len(consommation)-1,len(montant)-1
flag=0
somme=0
if len(consommation)<=len(titles)/2:
    while((min(i1,i2,i3,i4,i5)>-1)):
        if(i1 % 2==1):
            #print('working1')
            i1-=1
            i2-=1
            somme+=float(montant[i5])
            i5-=1
        else:
            #print('working2')
            s_title.append(titles[i1])
            s_month.append(month[i2])
            s_indexe.append(indexe[i3])
            s_consommation.append(consommation[i4])
            somme+=float(montant[i5])
            s_montant.append(somme)
            somme=0
            i1,i2,i3,i4,i5 = [v - 1 for v in (i1,i2,i3,i4,i5)]
else:
    while((min(i1,i2,i3,i4)>-1)and(i5> -2)):
        if ("قسط" in titles[i1]):
            flag = 1
            somme += float(montant[i5])
            i1-=1
            i5-=1
            i4-=1
        else :
            if (flag == 0):
                s_title.append(titles[i1])
                s_month.append(month[i2])
                s_indexe.append(indexe[i3])
                s_consommation.append(consommation[i4])
                s_montant.append(montant[i5])
                i5-=1
            else  :
                s_title.append(titles[i1])
                s_month.append(month[i2])
                s_indexe.append(indexe[i3])
                s_consommation.append(consommation[i4])
                s_montant.append(somme)
                somme=0
                flag=0
            i1,i2,i3,i4 = [v - 1 for v in (i1,i2,i3,i4)]


In [49]:
len(titles)

2

In [190]:
df = pd.DataFrame(list(zip(s_title,s_month,s_indexe,s_consommation,s_montant)),columns =['Type', 'Months', 'NV indexe','consommation','montant'])
df['n_facture']=n_facture
df['reference']=mois_ref[1]
df['date']=mois_ref[0]
df

,Type,Months,NV indexe,consommation,montant,n_facture,reference,date
0,*5 33 643155,006,error,21557,8433.864,834102022404,588469060,10/2022


In [30]:
montant

['312.000', '192.000', '532.800', '71.162', '1699.558']

In [133]:
tt=cv2.imread('mont.jpg')
res_keras=[]
predictions = pipeline.recognize([tt])
for prediction in predictions[0]:
    print(prediction[0])
    res_keras.append(prediction[0])

492
100


In [95]:
res_keras

['4182', '1600', '800', '1600', '182', '2702']

In [90]:
predictions[0][4][0]

'182'

In [85]:
prediction[1]

array([[         59,         106],
       [         97,         106],
       [         97,         121],
       [         59,         121]], dtype=float32)

In [66]:
ocr_im='month.jpg'
res_keras=[]
res_kerass=''
tt=cv2.imread(ocr_im)
res_tes=pytesseract.image_to_string(ocr_im,config=config)
res_tes=prepare(res_tes,'tab')
print(res_tes)
res_opus=textract.process(ocr_im,method='ocropus')
res_opus=res_opus.decode("utf-8")
res_opus=prepare(res_opus,'tab')
print(res_opus)
predictions = pipeline.recognize([tt])
for prediction in predictions[0]:
    res_keras.append(prepare(prediction[0],'tab'))
    res_kerass=' '.join(res_keras)
print(res_kerass)

r

p02


In [125]:
titles=perform_ocr('month.jpg','en')

il

002 0q2
st1: il st2: 012.3456789.. st3: 002 0q2


نأءة42572729 ناءة42 220921540 10


In [156]:
cv2.imwrite('exe.jpg', enhanced_img)
titles=perform_ocr('exe.jpg','ar')

نأءة 42 572729

نلاءة 42

ا 0 22092154
10

/

['نأءة 42 572729', 'نلاءة 42', 'ا 0 22092154', '10']
